In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import pygal
import time
import swifter

from collections import Counter
from matplotlib import pyplot as plt
from IPython.display import SVG, HTML, display

/home/tony/source/recsys-2019/.venv/lib/python3.7/site-packages/tqdm/autonotebook/__init__.py:18: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  " (e.g. in jupyter console)", TqdmExperimentalWarning)


In [2]:
df = pd.read_csv('../data/train.csv')

In [3]:
df.head()

,user_id,session_id,timestamp,step,action_type,reference,platform,city,device,current_filters,impressions,prices
0,00RL8Z82B2Z1,aff3928535f48,1541037460,1,search for poi,Newtown,AU,"Sydney, Australia",mobile,NaN,NaN,NaN
1,00RL8Z82B2Z1,aff3928535f48,1541037522,2,interaction item image,666856,AU,"Sydney, Australia",mobile,NaN,NaN,NaN
2,00RL8Z82B2Z1,aff3928535f48,1541037522,3,interaction item image,666856,AU,"Sydney, Australia",mobile,NaN,NaN,NaN
3,00RL8Z82B2Z1,aff3928535f48,1541037532,4,interaction item image,666856,AU,"Sydney, Australia",mobile,NaN,NaN,NaN
4,00RL8Z82B2Z1,aff3928535f48,1541037532,5,interaction item image,109038,AU,"Sydney, Australia",mobile,NaN,NaN,NaN


In [4]:
# only 1/15 of the interactions have filters applied
df.count()

user_id            15932992
session_id         15932992
timestamp          15932992
step               15932992
action_type        15932992
reference          15932992
platform           15932992
city               15932992
device             15932992
current_filters     1153112
impressions         1586586
prices              1586586
dtype: int64

In [5]:
# split_timestamp = pd.Timestamp(year=2018, month=11, day=5, hour=16, minute=52, second=28)
split_timestamp = pd.Timestamp(year=2018, month=11, day=6)
split_time =time.mktime(split_timestamp.timetuple()) # Unix epoch time

print(split_timestamp)
print(split_time)

2018-11-06 00:00:00
1541455200.0


In [6]:
df_train = df[df['timestamp'] < split_time]
df_validation = df[df['timestamp'] >= split_time]

In [7]:
print('train set size: {} ({})'.format(len(df_train), len(df_train) / len(df)))
print('validation set size: {} ({})'.format(len(df_validation), len(df_validation) / len(df)))

train set size: 12838797 (0.8057995008093898)
validation set size: 3094195 (0.19420049919061028)


In [8]:
df_train.to_csv('../data/train_split.csv', index=False)

In [9]:
df_validation.to_csv('../data/validation_split_truth.csv', index=False)

In [10]:
df_max_step_per_session = df_validation.groupby('session_id').apply(lambda session: session['step'].max())
df_max_step_per_session

session_id
000056cd97ce2      4
00011005327e1     29
0001fe0007954      8
000200da92d17      2
00027518c66ff     16
00028291d1704      1
0002ccee5a980      3
0002d9458ab1c      3
00030305c6aa7     18
00034fdbd1569      4
0003786b0a0d3      5
00038b7b71f53      2
00040aa1d0a4d     73
000457a2bec47      3
000476b04ccbe      3
00050dbe2a23e      3
0005138e57ff8      4
0005e5262fced      1
0005f3dacebc3      1
000667c53544d     31
0006855fc2acc     84
0006c25fcfd9e      8
00071e4fc649b      4
00077ca8b8ced      3
00079f55efc6f     33
0007d46ab5466     29
00085323c032e      3
0008928a1e2ff      1
0008e1891d8a3      2
0008e1b9de01b      4
                ... 
fff6fba73b9c3      1
fff70858d8783      3
fff800dda20f2      1
fff8bc05ad578      4
fff9122df42f6      1
fff963318fe06     20
fff9b10de9e0f      7
fff9c8fa8e51a      1
fff9f1d57a65e      7
fffa260a1e12b     68
fffa8e6f89f3b      1
fffae81f8eb8e    167
fffbce67be17c      1
fffbef76d20f8     80
fffc0720e963c      1
fffc33840a591     34
ff

In [11]:
def delete_last_step_clickout_reference(action):
    max_step = df_max_step_per_session[action['session_id']]
    return None if action['step'] == max_step and action['action_type'] == "clickout item" else action['reference']

In [12]:
df_validation['reference'] = df_validation.apply(delete_last_step_clickout_reference, axis=1)

/home/tony/source/recsys-2019/.venv/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [13]:
num_clickouts_to_predict = len(df_validation[(df_validation['action_type'] == 'clickout item') & (df_validation['reference'].isnull())])
num_clickouts_to_predict

124263

In [14]:
num_clickouts_to_predict / len(df_validation)

0.04016004162633577

In [15]:
df_validation.to_csv('../data/validation.csv', index=False)